In [1]:
import sys
import os
from scipy import linalg
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
sys.path.append("..")

from utils.camera_utils import *

In [2]:
run_name = "RUN4"

spiegel_calib = ["spiegel1.mp4", "spiegel3.mp4", "spiegel4.mp4", "spiegel5.mp4", "spiegel6.mp4", "spiegel7.mp4", "spiegel8.mp4",
                "spiegel9.mp4", "spiegel10.mp4"]
front_calib = ["front1.mp4", "front2.mp4", "front3.mp4", "front4.mp4", "front5.mp4", "front6.mp4", "front7.mp4", "front8.mp4", 
              "front9.mp4", "front10.mp4", "front11.mp4"]

stereo_calib = ["stereo1.mp4", "stereo2.mp4", "stereo3.mp4", "stereo4.mp4", "stereo5.mp4", "stereo6.mp4", "stereo7.mp4", 
                "stereo8.mp4", "stereo9.mp4", "stereo10.mp4", "stereo11.mp4", "stereo12.mp4"]

stereo_vali = ["stereo3.mp4", "stereo4.mp4", "stereo5.mp4"]

In [3]:
calibration_dir = f"../videos/{run_name}/calibration"
detection_dir = f"../videos/{run_name}/detection"
detection_files = os.listdir(detection_dir)
sC = stereoCamera(name=run_name, camera_size={0: (360, 180), 1: (360, 180)})
vL = videoLoader()


# Camera Positions

In [4]:
vL.load_video(os.path.join(detection_dir, detection_files[-1]))
frame = vL[10]
# 0 = mirror-cam 1 = frontal-cam
scaling_factor = 1 # image gets resized by this factor

sC.set_anchor_point(frame, 0, scaling_factor)
sC.set_anchor_point(frame, 1, scaling_factor)

# Displaying the camera regions for evaluation
for frame in vL[:100]:
    frame = sC.draw_camera_region(frame)
    cv2.imshow("Frame", frame)
    frame0, frame1 = sC(frame)
    cv2.waitKey(10)
cv2.destroyAllWindows()


# Mirror Camera Calibration

In [ ]:
frames = []
for file in spiegel_calib:
    vL.load_video(os.path.join(calibration_dir, file))
    video_section = vL[100:110]
    if len(video_section) > 0:
        frames.append(video_section)
    
sC.calibrate(frames,0, rows=8, columns=10, fallback_manual=True, image_scaling=2)

# Frontal Camera Calibration

In [ ]:


frames = []
for file in front_calib:
    vL.load_video(os.path.join(calibration_dir, file))
    frames.append(vL[100:110])
      
sC.calibrate(frames,1, rows=8, columns=10, fallback_manual=True, image_scaling=1.9)



In [ ]:
# Just for debugging
sC.save_to_yaml(f"{run_name} no stereo.yaml")

# Stereo Calibration

In [5]:
# Just for debugging

sC.load_from_yaml(f"{run_name} no stereo.yaml")

In [6]:
frames = []

for file in stereo_calib[0:]:
    print(os.path.join(calibration_dir, file))
    vL.load_video(os.path.join(calibration_dir, file))
    frames.append(vL[100:110])
opip1ip2 = None

# for testing different stereocalibration_flags without labeling the images new
#sC.load_from_yaml(f"{run_name} no stereo.yaml")
#with open("temp.pk", "rb") as f:
#    opip1ip2 = pk.load(f)

objectpoints, imgpoints_1, imgpoints_2, images = sC.stereo_calibrate(
    frames[:], 
    rows=8, 
    columns=10, 
    undistort=False, 
    opip1ip2=opip1ip2, 
    stereocalibration_flags=cv2.CALIB_USE_INTRINSIC_GUESS,
    image_scaling=3, 
    fallback_manual=True)
# cv2.CALIB_USE_INTRINSIC_GUESS, cv2.CALIB_FIX_INTRINSIC
# cv2.CALIB_FIX_K1 | cv2.CALIB_FIX_K2 | cv2.CALIB_FIX_K3 | cv2.CALIB_FIX_K4 | cv2.CALIB_FIX_K5 | cv2.CALIB_FIX_K6

../videos/RUN4/calibration/stereo1.mp4
../videos/RUN4/calibration/stereo2.mp4
../videos/RUN4/calibration/stereo3.mp4
../videos/RUN4/calibration/stereo4.mp4
../videos/RUN4/calibration/stereo5.mp4
../videos/RUN4/calibration/stereo6.mp4
../videos/RUN4/calibration/stereo7.mp4
../videos/RUN4/calibration/stereo8.mp4
../videos/RUN4/calibration/stereo9.mp4
../videos/RUN4/calibration/stereo10.mp4
../videos/RUN4/calibration/stereo11.mp4
../videos/RUN4/calibration/stereo12.mp4
Stereo-calibration error: 0.16002955712792535
Translation Matrix: [[-0.08341221]
 [-0.46210554]
 [ 0.45100709]]
Rotation Matrix: [[-0.98153114 -0.17024009 -0.0872636 ]
 [-0.13476321  0.29155586  0.94701323]
 [-0.1357774   0.9412829  -0.30911326]]
New Camera Matrix Mirror: [[824.9606694    0.         278.97450919]
 [  0.         893.82423828 167.50875544]
 [  0.           0.           1.        ]]
New Distortion Front [[-2.12826887e+00  3.03581863e+01 -3.15022137e-02 -9.98816977e-03
  -1.93186369e+02]]


# Save Calibration to YAML

In [7]:
sC.save_to_yaml(f"{run_name}.yaml")

sC.save_to_yaml(f"{run_name}_exp.yaml")

In [ ]:
sC.load_from_yaml(f"{run_name}.yaml")